# Normalization pipeline

Die Quelldaten sind aus meinem ersten RAG-Projekt und wurden mit fruendlicher Genehmigung von einer Webseite gescrappt.

**Was zu tun ist:**
- Daten von Marketingaussagen und Firmengeschichte befreien
- Die Specs normalisieren

Das meiste werde ich mit einem LLM machen. Die Ausgangsdaten werden stichprobenartig kontrolliert. Ziel ist ein funktionierendes und korrektes retrival, welches nicht zwingend mit der realen Welt (korrekte technische Daten z.B.) übereinstimmen muss. In einem Realworldprojekt ist das selbstverständlich etwas anderes.

Wärend der Bereinigung werden die Daten in Specs und Descriptions aufgeteilt.

In [4]:
import os
import json
import time
import pandas as pd
from tqdm import tqdm
from mistralai import Mistral
from dotenv import load_dotenv

load_dotenv()

df = pd.read_json('./products_raw.json')

api_key = os.getenv('MISTRAL_API_KEY')
model = 'mistral-medium-2508'
client = Mistral(api_key=api_key)

descs_promt = open('./descs_agent.md', 'r').read()
specs_promt = open('./specs_agent.md', 'r').read()
summs_promt = open('./summs_agent.md').read()

descs_chunks = []
specs_chunks = []
summs_chunks = []

Die Beschreibungen werden über ein LLM bereinigt. Der Systempromt liegt als md-File vor. 

### Funktionen

- **agent_request**: Allgemeine Retrivalfunktion. Fallbackfunktion (wegen fehlenden Specs integriert)
- **format_spec**: Bereitet Embeddingdocs für Specs auf

In [5]:
def agent_request(promt, content, fallback):

    if content == '[]': content = fallback
    
    response = client.chat.complete(
        model = model,
        messages = [
            {
                'role': 'system',
                'content': promt
            },
            {
                'role': 'user',
                'content': content,
            }
        ],
        response_format = {"type": "json_object"}
    )

    return response

def format_spec(group_name, specs_list):
    if not specs_list:
        return ''
    
    if '-' in group_name:
        label, unit = group_name.rsplit('-', 1)
        header = f"{label} ({unit})"
    else:
        header = group_name
    
    spec_parts = []

    for key, value in specs_list.items():
        readable_key = key.replace('_', ' ')
        spec_parts.append(f"{readable_key}: {value}")

    return f"{header} - {', '.join(spec_parts)}"

# Erstellung der Chunks

Die Daten werden an ein LLM geliefert und wir erhalten nur die erzeugen Artefarkte. Diese werden in des Schema der Chunks integriert:

In [6]:
for index, row in tqdm(df.iterrows(), total=len(df)):

    summs_response = agent_request(summs_promt, json.dumps(row.to_dict(), ensure_ascii=False), False)
    descs_response = agent_request(descs_promt, row['description'], False)
    specs_response = agent_request(specs_promt, json.dumps(row['specs'], ensure_ascii=False), row['description'])

    summs_result = json.loads(summs_response.choices[0].message.content)                              
    descs_result = json.loads(descs_response.choices[0].message.content)
    specs_result = json.loads(specs_response.choices[0].message.content)

    # Debug only
    # print(summs_result)
    # print(descs_result)
    # print(specs_result)

    category = summs_result['category']
    manufacturer = summs_result['manufacturer']
    title = summs_result['title']

    product_metadata = {
        'product_id': row['id'],
        'product_title': title,
        'product_url': row['url'],
        'product_category': category,
        'product_manufacturer': manufacturer,
        'product_price': row['price'],
        'product_sku': row['order_number']
    }

    summs_chunks.append({
        'document': summs_result['summary'],
        'metadata': {
            'chunk_id': f"{row['id']}_summ_0",
            'chunk_type': 'description',
            **product_metadata
        }
    })

    for i, pararagraph in enumerate(descs_result):
        descs_chunks.append({
            'document': pararagraph,
            'metadata': {
                'chunk_id': f"{row['id']}_desc_{i}",
                'chunk_type': 'description',
                **product_metadata
            }
        })

    for i, (group_name, specs_list) in enumerate(specs_result.items()):
        if specs_list:
            specs_chunks.append({
                'document': format_spec(group_name, specs_list),
                'metadata': {
                    'chunk_id': f"{row['id']}_spec_{i}",
                    'chunk_type': 'specs',
                    **product_metadata
                }
            })

    # Testing only
    # if index == 65: break

summs_df = pd.DataFrame(summs_chunks)
descs_df = pd.DataFrame(descs_chunks)
specs_df = pd.DataFrame(specs_chunks)

100%|██████████| 151/151 [43:01<00:00, 17.10s/it]


## Quality Checks

Wir müssen zuletzt noch in die Chunks schauen ob sie die wichtigsten Rahmenbedinungen einhalten. Vor allem die Länge da die meisten Models eine beschränkte Anzahl an Tokes verarbeiten kann und die darf von den Chunks nicht überschritten werden.

In [7]:
def stats(df, treshhold, name):

    df['length'] = df['document'].str.len()
    small = df[df['length'] < treshhold]

    print(f'=== {name} ===')
    print(f"{df['length'].describe()}")
    print(f'TOTAL CHUNKS: {len(df)}')
    print(f'SHORT CHUNKS: {len(small)}')

    for idx, row in small.iterrows():
        
        print(f"{row['metadata']['product_title']}")
        print(f"Document: {row['document']}")
        print(f"Length: {row['length']}")
        print(f'--- --- ---')

    print('\n')


In [8]:
stats(summs_df, 200, 'Summs')
stats(descs_df, 200, 'Descriptions')
stats(specs_df, 100, 'Specs')

=== Summs ===
count     151.000000
mean      885.940397
std       161.116862
min       586.000000
25%       775.000000
50%       867.000000
75%       995.500000
max      1288.000000
Name: length, dtype: float64
TOTAL CHUNKS: 151
SHORT CHUNKS: 0


=== Descriptions ===
count     909.000000
mean      353.537954
std       112.353517
min        87.000000
25%       281.000000
50%       341.000000
75%       414.000000
max      1300.000000
Name: length, dtype: float64
TOTAL CHUNKS: 909
SHORT CHUNKS: 45
SFFsg 5501 Performance
Document: Die Geräte sind besonders energieeffizient und entsprechen den Sicherheitsstandards nach IEC 61010-2-011 sowie der ATEX-Richtlinie (SRFfg/SFFfg).
Length: 145
--- --- ---
BL-720 PRO-ACTIVE
Document: Zusätzlich verfügt das Gerät über eine RS 485 Schnittstelle zur Anbindung an eine PC-gestützte Temperaturdokumentation. Eine Alarmerweiterung über einen potentialfreien Kontakt ist ebenfalls möglich.
Length: 199
--- --- ---
HC 502
Document: Der Dometic HC 502 FS ist au

An dieser Stelle wurden weitere Verbesserungen an den Promts vorgenommen und getestet... Mit den Specs muss noch eine andere Strategie ausgedacht werden. Jetzt erstmal mit den kleinen Chunks probieren. 

## Speichern

In [9]:
summs_df.to_json('summs_chunks.jsonl', orient='records', lines=True, force_ascii=False)
descs_df.to_json('descs_chunks.jsonl', orient='records', lines=True, force_ascii=False)
specs_df.to_json('specs_chunks.jsonl', orient='records', lines=True, force_ascii=False)